# Detector de FakeNews
Autor: Edson Adriano Maravalho Avelar

Resume: O objetivo deste projeto é criar um detector de fakenews utilizando NLP - Natural Language Processing. Por hora esse projeto será feito em inglês, mas futuramente a ideia é adaptar para o português, quando sair um dataset confiável de fakenews nesse idioma. 

O modelo resultante será exportado e colocado em produção usando o Flask. Uma API Pública será criada para tal, ela poderá ser consumida por qualquer pessoa. 
Link da API:


In [ ]:
%reset -f
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
true_data = pd.read_csv("/kaggle/input/onion-notonion-datasets/True.csv",parse_dates=['date'])
fake_data = pd.read_csv("/kaggle/input/onion-notonion-datasets/Fake.csv", parse_dates=['date'])

true_data['y'] = 1
fake_data['y'] = 0

In [ ]:
true_data.head(5)

In [ ]:
true_data.sort_values("date")

In [ ]:
fake_data.head(5)

## Problemas com o date do Fake Data
Existem dados que não são timestamp na coluna date

In [ ]:
import datetime


for i, date in enumerate(fake_data['date']):
    try:
        fake_data.iloc[i,fake_data.columns.get_loc('date')] =  pd.Timestamp(date) 
    except:
        fake_data.iloc[i,fake_data.columns.get_loc('date')] = fake_data['date'].mode()[0]
fake_data['date'] = pd.to_datetime(fake_data['date']) 

In [ ]:
data = pd.concat( [true_data, fake_data],ignore_index=True).sort_values('date')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
plot = data.groupby( data['date'].dt.date )['y'].mean().reset_index()

plt.figure(figsize=(10,5))
sns.lineplot( plot.date, plot.y )
plt.title(" Ocorrência de Fakenews por ano ")
plt.show()

## Percebe-se pelo gráfico que que as ocorências de fakenews começaram a partir de 2016 vão até início de 2018. 

In [ ]:
data2 = data[ data['date']  > '2016-01-01' ]
data2 = data2[ data2['date']  < '2018-01-01' ]

In [ ]:
data2.shape

In [ ]:
plot = data2.groupby( data2['date'].dt.date )['y'].mean().reset_index()
plt.figure(figsize=(10,5))
sns.lineplot( plot.date, plot.y )
plt.title(" Ocorrência de Fakenews por ano ")
plt.show()

## Preprocessamento

In [ ]:
data2['title_pre'] = 0
data2['title_pre'] = data2['title'].str.lower().str.replace(r'[^\w\a]+',' ')
data2['title_pre'].head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

from sklearn.model_selection import train_test_split

In [ ]:
x = data2['title_pre'].copy()
y = data2['y'].copy()
x.shape,y.shape

In [ ]:
x.head(5)

In [ ]:
y.head(5)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size =0.3)

# Baseline

In [ ]:
base = np.ones(xtest.shape[0]) *ytrain.mean()
base.mean()

In [ ]:
log_loss(ytest, base)

In [ ]:
mdl = make_pipeline(TfidfVectorizer(min_df=1, ngram_range=(1,1)), RandomForestClassifier(n_estimators=1000, n_jobs=6, random_state = 0)   )
mdl.fit(xtrain,ytrain)
p = mdl.predict_proba(xtest)[:,1]
log_loss(ytest,p)

In [ ]:
mdl = make_pipeline(TfidfVectorizer(min_df=1, ngram_range=(1,1)), LogisticRegression(C=20.)   )
mdl.fit(xtrain,ytrain)
p = mdl.predict_proba(xtest)[:,1]
log_loss(ytest,p)

In [ ]:
mdl = make_pipeline(TfidfVectorizer(min_df=1, ngram_range=(1,1)), LogisticRegression(C=20.)   )
mdl.fit(x,y)
p = mdl.predict_proba(xtest)[:,1]
log_loss(ytest,p)

In [ ]:
import joblib as jb
jb.dump(mdl,"mdl.pkl.z")